<a href="https://colab.research.google.com/github/dkle91/Hyperparameter_Optimization_for_PINNs_using_GA/blob/main/Wave_equation_Grid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np
import scipy.io
from keras import models 
import matplotlib.pyplot as plt
from time import time
from mpl_toolkits.mplot3d import Axes3D
from google.colab import files

# Set data type
DTYPE='float32'
tf.keras.backend.set_floatx(DTYPE)

# Set constants
pi = tf.constant(np.pi, dtype=DTYPE)
c2 = 1

# Define initial & boundary conditions
def u_0(x):
    return tf.sin(pi*x)
def v_t_0(x):
    return -pi*tf.sin(pi*x)

# Define residual of the PDE
def fun_r(t, x, u, u_tt, u_xx):
    return u_tt - c2 * u_xx

# Define exact solution function
def fun_exact(t, x):
    return tf.sin(pi*x)*tf.cos(pi*t) - tf.sin(pi*x)*tf.sin(pi*t)


# Set boundary
tmin = 0.
tmax = 1.
xmin = 0.
xmax = 1.

# Lower bounds
lb = tf.constant([tmin, xmin], dtype=DTYPE)
# Upper bounds
ub = tf.constant([tmax, xmax], dtype=DTYPE)

# Define t,x for evaluation and exact solutions
num_eval = 100
t_eval = tf.linspace(lb[0], ub[0], num_eval)
x_eval = tf.linspace(lb[1], ub[1], num_eval)
T_eval,X_eval = np.meshgrid(t_eval, x_eval) # make X_train [[x1 x2 x3 ... xn], [x1 x2 x3 ...xn],...(Nt number)....[x1 x2 x3 ... xn]] and make T_train [[t1 t1 t1 ... t1], [t2 t2 t2 ...t2],...(Nx number)....[tn tn tn ... tn]]
teval = T_eval.ravel() # make it one row
teval = tf.reshape(tf.convert_to_tensor(teval),shape=(-1,1)) # Convert it to tensor and make it one column
xeval = X_eval.ravel() # make it one row
xeval = tf.reshape(tf.convert_to_tensor(xeval),shape=(-1,1)) # Convert it to tensor and make it one column

# Set random seed for reproducible results
tf.random.set_seed(0)

# Result management
Result = np.ones((16384, 11))*999999
i= 0

# a. Number of train points in domain: 0-50, 1-100, 2-150, 3-200
num_train_domain = [50, 100, 150, 200]
# b. Activation function: 0-tanh, 1-sigmoid, 2-Relu, 3-selu
activation = ['tanh', 'sigmoid', 'relu', 'selu']
# c. Optimizer: 0-Adam, 1-SGD, 2-RMSprop, 3-Adadelta
optimizer = ['Adam', 'SGD', 'RMSprop', 'Adadelta']
# d. Learning rate: 0-1e-2, 1-5e-3, 2-1e-3, 3-5e-4
learn_rate = [1e-02, 5e-03, 1e-03, 5e-4]
# e. Number of epochs: 0-2500, 1-5000, 2-7500, 3-10000
N_epoch = [2500, 5000, 7500, 10000]
# f. Number of neurons: 0-50, 1-100, 2-150, 3-20
num_neurons_per_layer = [5, 10, 15, 20]
# g. Number of layers: 0-1, 1-2, 2-3, 3-4
num_hidden_layers = [1, 2, 3, 4]


def init_model(num_hidden_layers, num_neurons_per_layer):
    # Initialize a feedforward neural network
    model = tf.keras.Sequential()

    # Input is two-dimensional (time + one spatial dimension)
    model.add(tf.keras.Input(2))

    # Introduce a scaling layer to map input to [lb, ub]
    scaling_layer = tf.keras.layers.Lambda(
                lambda x: 2.0*(x - lb)/(ub - lb) - 1.0)
    model.add(scaling_layer)

    # Append hidden layers
    for _ in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons_per_layer,
            activation=tf.keras.activations.get(activation_func),
            kernel_initializer='glorot_normal'))

    # Output is one-dimensional
    model.add(tf.keras.layers.Dense(1))
    
    return model

def get_r(model, X_r):
    # A tf.GradientTape is used to compute derivatives in TensorFlow
    with tf.GradientTape(persistent=True) as tape:
        # Split t and x to compute partial derivatives
        t, x = X_r[:, 0:1], X_r[:,1:2]

        # Variables t and x are watched during tape
        # to compute derivatives u_t and u_x
        tape.watch(t)
        tape.watch(x)

        # Determine residual 
        u = (1 - t**2)*u_0(x) + t*v_t_0(x) + x*(1-x)*t**2*model(tf.stack([t[:,0], x[:,0]], axis=1))

        # Compute gradient u_x within the GradientTape
        # since we need second derivatives
        u_x = tape.gradient(u, x)        
        u_t = tape.gradient(u, t)
    u_xx = tape.gradient(u_x, x)
    u_tt = tape.gradient(u_t, t)
    del tape
    return fun_r(t, x, u, u_tt, u_xx)

def compute_loss(model, X_r):
    # Compute phi^r
    r = get_r(model, X_r)
    phi_r = tf.reduce_mean(tf.square(r))
    # Initialize loss
    loss = phi_r
    return loss

def get_grad(model, X_r):
    with tf.GradientTape(persistent=True) as tape:
        # This tape is for derivatives with
        # respect to trainable variables
        tape.watch(model.trainable_variables)
        loss = compute_loss(model, X_r)
    g = tape.gradient(loss, model.trainable_variables)
    del tape
    return loss, g

# Calculation for exact solutions
exact = fun_exact(teval, xeval)
Exact = np.reshape(exact,(num_eval, num_eval))
Teval, Xeval = np.meshgrid(t_eval, x_eval)
Eval_grid = np.vstack([Teval.flatten(),Xeval.flatten()]).T

# Surface plot of exact solution u(t,x)
#fig = plt.figure(figsize=(9,6))
#ax = fig.add_subplot(111, projection='3d')
#ax.plot_surface(Teval, Xeval, Exact, cmap='jet');
#ax.view_init(35,35)
#ax.set_xlabel('$t$')
#ax.set_ylabel('$x$')
#ax.set_zlabel('$u(t,x)$')
#ax.set_title('Exact solution of wave equation \n');
#plt.savefig('Exact solution of wave equation.jpeg', bbox_inches='tight', dpi=300);

# Initialize model aka u_\theta
for a in range(len(num_train_domain)):
    N_r = num_train_domain[a]
    # Draw uniformly sampled collocation points
    t_r = tf.random.uniform((N_r,1), lb[0], ub[0], dtype=DTYPE)
    x_r = tf.random.uniform((N_r,1), lb[1], ub[1], dtype=DTYPE)
    X_r = tf.concat([t_r, x_r], axis=1)
    # Collect boundary and inital data in lists
    #X_data = [X_0, X_b]
    #u_data = [u_0, u_b]
    #fig = plt.figure(figsize=(9,6))
    #plt.scatter(t_r, x_r, c='black', marker='.',alpha=0.3)
    #plt.xlabel('$t$')
    #plt.ylabel('$x$')
    #plt.title('Positions of train points');
    #plt.savefig('Positions of train points.jpeg', bbox_inches='tight', dpi=300)
    for b in range(len(activation)):
      activation_func = activation[b]
      for c in range(len(optimizer)):
        for d in range(len(learn_rate)):
          learning_rate_func = learn_rate[d]
          for e in range(len(N_epoch)):
            N_epoch_func = N_epoch[e]
            for f in range(len(num_neurons_per_layer)):
              num_neurons_per_layer_func = num_neurons_per_layer[f]
              for g in range(len(num_hidden_layers)):
                    Result[i,0] = i + 0
                    Result[i,1] = a
                    Result[i,2] = b
                    Result[i,3] = c
                    Result[i,4] = d 
                    Result[i,5] = e 
                    Result[i,6] = f 
                    Result[i,7] = g 
                    num_hidden_layers_func = num_hidden_layers[g]
                    # Define one training step as a TensorFlow function to increase speed of training
                    @tf.function
                    def train_step(model):
                      loss, grad_theta = get_grad(model, X_r)
                      # Perform gradient descent step
                      optim.apply_gradients(zip(grad_theta, model.trainable_variables))
                      return loss
                    model = init_model(num_hidden_layers_func,num_neurons_per_layer_func)
                    # Choose the optimizer
                    if optimizer[c] == 'Adam':
                      optim = tf.keras.optimizers.Adam(learning_rate_func)
                    if optimizer[c] == 'SGD':
                      optim = tf.keras.optimizers.SGD(learning_rate_func)
                    if optimizer[c] == 'RMSprop':
                      optim = tf.keras.optimizers.RMSprop(learning_rate_func)
                    if optimizer[c] == 'Adadelta':
                      optim = tf.keras.optimizers.Adadelta(learning_rate_func)
                    #hist = []
                    # Start timer
                    t0 = time()
                    for k in range(N_epoch_func+1):
                        loss = train_step(model)
                        # Append current loss to hist
                        #hist.append(loss.numpy())
                        # Output current loss after 50 iterates
                        #if k%50 == 0:
                          #print('It {:05d}: loss = {:10.8e}'.format(k,loss))
                    Loss_final = loss.numpy()
                    # Print cases number
                    print('Cases number: ',i)
                    # Print computation time and Loss
                    Time_final = time()-t0
                    print('Computation time:(seconds) ',Time_final)
                    print('Final loss: ',Loss_final)

                    # Evaluation predicted solutions
                    ueval = (1 - teval**2)*u_0(xeval) + teval*v_t_0(xeval) + xeval*(1-xeval)*teval**2*model(tf.cast(Eval_grid,DTYPE))
                    # Reshape ueval
                    Ueval = ueval.numpy().reshape(num_eval,num_eval)
                    # Error evaluation
                    Error = np.abs(Ueval - Exact)
                    L2_error = np.linalg.norm(Error,2)/np.linalg.norm(Exact,2)
                    print('Relative L2 error: ',L2_error)

                    # Assign reuslts
                    Result[i,8]= Time_final
                    Result[i,9]= Loss_final
                    Result[i,10]= L2_error
                    i +=1
                    # Plot of loss function
                    #fig = plt.figure(figsize=(9,6))
                    #ax = fig.add_subplot(111)
                    #ax.semilogy(range(len(hist)), hist,'k-')
                    #ax.set_xlabel('$n_{epoch}$')
                    #ax.set_ylabel('$Loss$');
                    #ax.set_title('Loss of the trained neural networks \n');
                    #plt.savefig('Loss of the trained neural networks.jpeg', bbox_inches='tight', dpi=300);

                    # Surface plot of predicted solution u(t,x)
                    #fig = plt.figure(figsize=(9,6))
                    #ax = fig.add_subplot(111, projection='3d')
                    #ax.plot_surface(Teval, Xeval, Ueval, cmap='jet');
                    #ax.view_init(35,35)
                    #ax.set_xlabel('$t$')
                    #ax.set_ylabel('$x$')
                    #ax.set_zlabel('$u(t,x)$')
                    #ax.set_title('Predicted solution of wave equation \n');
                    #plt.savefig('Predicted solution of wave equation.jpeg', bbox_inches='tight', dpi=300);

                    # Surface plot of absolute error
                    #fig = plt.figure(figsize=(9,6))
                    #ax = fig.add_subplot(111, projection='3d')
                    #ax.plot_surface(Teval, Xeval, Error, cmap='jet');
                    #ax.view_init(35,35)
                    #ax.set_xlabel('$t$')
                    #ax.set_ylabel('$x$')
                    #ax.set_zlabel('$u_\\theta(t,x)$')
                    #ax.set_title('Difference between predicted and exact solutions of wave equation \n');
                    #plt.savefig('Difference between predicted and exact solutions of wave equation.jpeg', bbox_inches='tight', dpi=300);

np.savetxt('Wave-Summary.csv', Result, delimiter=',')
#Download result file
files.download('Wave-Summary.csv') 
